In [1]:
import sys
from netgen.csg import CSGeometry, OrthoBrick, Pnt
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi

# -------------------------------- PARAMETERS ---------------------------------
# Mesh diameter
maxh = 0.6
# Bisect cut elements of (initial) mesh
n_cut_ref = 2
# Polynomial order of FE space
order = 2

# Problem parameters
reac_cf = 1
diff_cf = 1

# Geometry
cube = CSGeometry()
cube.Add(OrthoBrick(Pnt(-1.41, -1.41, -1.41), Pnt(1.41, 1.41, 1.41)))
mesh = Mesh(cube.GenerateMesh(maxh=maxh, quad_dominated=False))

levelset = sqrt(x**2 + y**2 + z**2) - 1
exact = sin(pi * z)
coef_f = (sin(pi * z) * (diff_cf * pi * pi * (1 - z * z) + reac_cf)
          + diff_cf * cos(pi * z) * 2 * pi * z)

# ----------------------------------- MAIN ------------------------------------
# Preliminary refinements
for i in range(n_cut_ref):
    lsetp1 = GridFunction(H1(mesh, order=1))
    InterpolateToP1(levelset, lsetp1)
    RefineAtLevelSet(lsetp1)
    mesh.Refine()

# Class to compute the mesh transformation needed for higher order accuracy
#  * order: order of the mesh deformation function
#  * threshold: barrier for maximum deformation (to ensure shape regularity)
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=1000,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lset_approx = lsetmeshadap.lset_p1

# Background FESpace
Vh = H1(mesh, order=order, dirichlet=[])

ci = CutInfo(mesh, lset_approx)
ba_IF = ci.GetElementsOfType(IF)

importing ngsxfem-2.1.2504


In [5]:
DrawDC(lset_approx,-1,1,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [3]:
Draw(BitArrayCF(ci.GetElementsOfType(NEG)),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
"""
These examples show a number of pre-implemented 3d level set geometries,
computes the distance between the smooth level set and the (deformed) P1
approximation as well as the order of convergence thereof.

References:
-----------
* All concepts that are used here are explained in the jupyter-tuorials
  `basics` and `intlset`.
"""

# ------------------------------ LOAD LIBRARIES -------------------------------
from netgen.csg import CSGeometry
from ngsolve import *
from ngsolve.internal import *

from xfem import *
from xfem.lsetcurv import *
from xfem.utils import LevelsetExamples, BoundingBoxes

import sys

ngsglobals.msg_level = 2

# -------------------------------- PARAMETERS ---------------------------------
# Initial mesh size
maxh = 0.5
# Mesh deformation order
order = 2
# Refine cut elements
maxreflvl = 4

# ----------------------------------- MAIN ------------------------------------
# Main loop
for lsetgeom in ["cheese", "torus", "dziukelliott", "dziuk88", "sphere"]:
    print('Geometry: ', lsetgeom)

    geom = CSGeometry()
    geom.Add(BoundingBoxes[lsetgeom])
    mesh = Mesh(geom.GenerateMesh(maxh=maxh))
    levelset = LevelsetExamples[lsetgeom]

    lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=100,
                                          discontinuous_qn=True)

    Draw(levelset, mesh, "levelset")
    Draw(lsetmeshadap.deform, mesh, "deformation")
    Draw(lsetmeshadap.lset_p1, mesh, "levelset(P1)")

    distances = []
    for reflevel in range(maxreflvl):

        if (reflevel > 0):
            mesh.Refine()

        deformation = lsetmeshadap.CalcDeformation(levelset)
        distances.append(lsetmeshadap.CalcMaxDistance(levelset, deform=True))

        # Refine cut elements:
        RefineAtLevelSet(gf=lsetmeshadap.lset_p1)

        # Post-processing
        Redraw(blocking=True)
        eoc = [log(distances[i - 1] / distances[i]) / log(2)
               for i in range(1, len(distances))]
        print("distances = {}".format(distances))
        print("eoc = {}".format(eoc))

    if (hasattr(sys, 'ps1') or sys.flags.interactive
            or 'running_in_netgen' in dir()):
        input("press ENTER to go to next geometry (or finish)")


Geometry:  cheese
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 6
 Optimize Surface
 Surface 2 / 6
 Optimize Surface
 Surface 3 / 6
 Optimize Surface
 Surface 4 / 6
 Optimize Surface
 Surface 5 / 6
 Optimize Surface
 Surface 6 / 6
 Optimize Surface
 Delaunay meshing
 Volume Optimization


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

 SearchCorrespondingPoint:: did not converge 
distances = [1.3462298489554385]
eoc = []
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 SearchCorrespondingPoint:: did not converge 
 Mesh bisection
 Bisection done
distances = [1.3462298489554385, 0.9661163741385055]
eoc = [0.47865586459296233]
 Mesh bisection
 Bisection done
distances = [1.3462298489554385, 0.9661163741385055, 0.04394709559947518]
eoc = [0.47865586459296233, 4.458357252256012]
 Mesh bisection
 Bisection done
distances = [1.3462298489554385, 0.9661163741385055, 0.0439470